# Embeddings

Embedding is a very powerful idea. It allows a developer to create custom application constructs (such as an API), so that others can use it in their code, in addition to mixing these constructs with others, which could be built-in in the host language, or embedded in it by other embeddings and libraries.

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">

<h2>Terminology</h2>

</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">

<p>
We outline below the key terms we will be discussing in these notes: <br/>
<ol>
<li><b>Host Language:</b> the programming language in which developers create their embeddings. </li>
<li><b>Embeddings:</b> custom logic and abstraction create by implementers in some host language. An embedding may be a large or small piece of code, it may perform a single specific task, or provide a suite of functionality. It may be organized and shipped as a library, or could be part of a larger application. </li>
<li><b>Client Code:</b> code that uses the embedding. It could be written by the same developer as the embedding, or by external developers that use the embedding as a library. </li>
</ol>
</p>

<p>
We enumerate three categories of embeddings that we will study in these notes: <br/>
<ol>
<li> <b>Shallow Embeddings</b>. </li>
<li> <b>Deep Embeddings</b>. </li>
<li> <b>External Embeddings</b>. </li>
</ol>
</p>

<p>
In these notes, we will be studying these three categories of embedding through a case study. As well as discussing their conceptual definition and tradeoffs.
</p>
</div>

## Scenario

Our scenario is as follows. We are given some client code, that includes variables and expressions. However, we know that some of these variables are assigned complex expressions but are not used. Since python follows an eager evaluation strategy, these expressions will be evaluated, even if their values do not really get used.

We want to write an embedded library, that will automatically make sure that such expressions do not get evaluated.

There are several ways we can achieve this: (1) we can change python's evaluation semantics to use lazy evaluation, i.e. only evaluates an expression when it is used (2) we can automatically parse and transform any client code and remove such unused variables. We will implement both approaches in these notes. However, we will first make a quick detour to discuss embeddings conceptually.

# Shallow and Deep Embeddings

Shallow and Deep Embeddings are both mathematical concepts stemming from logic. However, we can interpret them in a computation way, so that they apply to common programming practice.

This ability to use logical concepts in programming (and vice-versa) is a consequence of the *[Curry-Howard Isomorphism](https://en.wikipedia.org/wiki/Curry%E2%80%93Howard_correspondence)*. At a high-level, it indicates that there is an equivalence between systems of logic and certain programming paradigm, with the more expressive logic systems being equivalent to more expressive programming paradigm.

We will begin by talking about shallow and deep embeddings in terms of logic, and concretize them in terms of programming via some examples.

_**For those who are interested in learning more**: This isomorphism shows a way that deduction rules in a logic can be transformed into **equivalent** programming expressions and vice-versa, and therefore establish a correspondance between proofs and programs, or between a logic and a programming language. The most basic form of this correspondance establishes a translation between rules of natural deduction in intuitionistic propositional logic and simply-typed lambda calculus._


## Embeddings in Logic

_Disclaimer: The following is background material that may be interesting to some. It hints at the connections between programming and logic. However, some of you may be more comfortable with looking at embeddings via programming instead. This is perfectly fine, because logic and programming are faces in the same coin. If you understand a concept in programming or in logic, your understanding will intuitively extend to the other system, even if you cannot express it in that system's terms. Feel free to focus on the variant that suits you best._

In logic, an embedding of one language or logic into another is called *[shallow](https://en.wiktionary.org/wiki/shallow_embedding)*, when the embedded logic is defined directly via the constructs of the host logic. In other words, the embedded logic's semantics is defined by a *syntactic* translation between its constructs and equivalent ones in the host logics. In some ways, you can think of the embedded language in this case as syntactic sugar that makes the host logic friendlier or easier to use in some domain.

Here is a very simple example: consider propositional logics L1 and L2. The first is made up of *logical and* and *logical negation*, while the second is made up of *logical and* and *logical or*. We can define a shallow embedding of L2 into L1 via the following transformation, which effectively extends L1 with a syntactic *logical or* operator:
1. logical and of both systems are equivalent.
2. logical or is defined as follows: x _or_ y := not (not x _and_ not y).

We can verify the correctness of this transformation by looking at its truth table. Note that the two columns in bold are the same:

| x | not x | y | not y | **x _or_ y** | not x _and_ not y | **not (not x _and_ not y)** |
|---|-------|---|-------|--------------|-------------------|-----------------------------|
| 1 | 0     | 1 | 0     | **1**        | 0                 | **1**                       |
| 0 | 1     | 0 | 1     | **1**        | 0                 | **1**                       |
| 1 | 0     | 1 | 0     | **1**        | 0                 | **1**                       |
| 0 | 1     | 0 | 1     | **0**        | 1                 | **0**                       |

Deep embeddings are more interesting. An embedding of a logic into another is called *[deep](https://en.wiktionary.org/wiki/deep_embedding)*, if the constructs or expressions of the host logic are used as data in the other. This means that we can use constructs from the host language to organically manipulate constructs of the embedded logic.

Both styles of embeddings do not make the host logic more *expressive*, since that logic already had the capability to express the embedded logic. However, it makes the host logic friendlier or easier to use for certain operations. For example, in our shallow embeddings above, we can use the new operator *logical or* directly, instead of having to write an equivalent but longer expression in L1.

Here are some more sophistacted examples in more expressive logics using Haskell: [example1](http://people.oregonstate.edu/~youngjef/posts-output/2018-05-05-dsl-embedding.html) and [example2](https://alessandrovermeulen.me/2013/07/13/the-difference-between-shallow-and-deep-embedding/).

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">

<h2> Embeddings in Programming </h2>

</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">

<p>
We can revisit the above definitions from a programming perspective. A <i>shallow</i> embedding involves using the constructs of the host programming language directly in order to define new helpful functionality. In a sense, any given function is a shallow embedding. Since you can think of a function as creating a new expression in the host language offering a new functionality defined using existing functionality.
</p>

<p>
In some languages, you can override operators (i.e. change the implementation of +) or define new ones (by modifying the syntax of the language). This does not make the shallow embedding deep, since at the bottom of the syntax sugar, there is a function that implements the embedded feature using host language constructs.
</p>

<p>
For example, below is an implementation of <i>logical or</i> in python, via the same transformation given in the previous section. This is a shallow embedding, since it is defined by using 'return', 'not', and 'and' only.
</p>

</div>

In [1]:
def logical_or(x, y):
    return not (not x and not y)

print(logical_or(True, True))
print(logical_or(True, False))
print(logical_or(False, True))
print(logical_or(False, False))

True
True
True
False


<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">

<p>
A <i>deep</i> embedding involves defining the embedded language constructs by representing them as a data-type, accompanied by interesting functionality and transformation. In object orient programming languages, you can think of each new embedded construct as a Class with methods and functions that operate on it, for example, to interpret/execute or transform/analyze the construct.
</p>

<p>
This provides more flexibility than a shallow embedding: (1) it allows users to extend or customize the new construct, by providing new functionality or overriding existing functionality (e.g. using inheritance) (2) it allows users to arbitrary mix the new constructs with other existing constructs via standard programming techniques. We will demonstrate this via an example in the next section.
</p>

<p>
Note that both embeddings do not make the programming language more expressive, since both embeddings are implemented using the host language. However, they can make the language friendier or easier to use for certain applications, or improve it in general. This is very similar to the rationale behind building libraries.
</p>

<p>
If you are interested in learning more, read this excellent short article on <a href="http://composition.al/blog/2015/06/02/embedding-deep-and-shallow/">embeddings</a>.
</p>

</div>

# Lazy Evaluation via Deep Embedding in Python

Remember our application scenario: we would like to find some way to automatically optimize client programs, by not executing useless statements.

Our first suggested approach is to extend python semantics to support lazy evaluation, so that an expression is only evaluated when it is needed. One way to implement this is to create a new python class (called Lazy below), that implements our supported operations in a lazy way: instead of evaluating the operation directly, a record of the operation is made and returned, and all recored operations are executed when their record is used.

Say we only want to support _print_, + and \*. We would override these operators in our Lazy class, and make them return a new Lazy instance, that stores the operands in its attributes. If that instance is used again with another operator, the same behavior occurs compositionally. Finally, when the instance is used (i.e. printed), we evaluate it in its entirety recursively.

Our Lazy class implementation below is a deep embedding. Since it represents constructs in our language (lazy + and * operators) as data: particularly as objects of type Lazy. It also provides a functionality to evaluate/execute the lazy operators.

In [2]:
from collections import namedtuple

# Represents a record of an operation, containing operation type (+ and *)
# the left operand, and the right operand.
BinOp = namedtuple('BinOp', ['op', 'left', 'right'])

class Lazy(object):
    # constructor: stores the passed value for future evaluation
    def __init__(self, value):
        self.value = value

    # override + and *
    def __add__(self, o):
        return self.__class__(BinOp('+', self, o))

    def __mul__(self, o):
        return self.__class__(BinOp('*', self, o))

    # print() calls __str__ internally
    def __str__(self):
        return str(self.eval())

    # evaluate our instance recursively:
    # if value is a number, return it as the value
    # if value is BinOp, evaluate left and right recursively
    #    then apply the operation on the two results
    def eval(self, debug=True):
        if debug: # for debugging
            print('Lazy expression:', self.inspect())

        if isinstance(self.value, int):
            return self.value
        else:
            ev1 = self.value.left.eval(debug=False)
            ev2 = self.value.right.eval(debug=False)
            if self.value.op == '+':
                return ev1 + ev2
            elif self.value.op == '*':
                return ev1 * ev2

    # pretty print the content of this instance to a string
    def inspect(self):
        if isinstance(self.value, int):
            return str(self.value)
        else:
            i1 = self.value.left.inspect()
            i2 = self.value.right.inspect()
            return '({}) {} ({})'.format(i1, self.value.op, i2)

Any client code can use our Lazy class now, by using it to wrap numeric values and variables, and then using the Lazy instances regularly. Note how the unused variable's expression is not evaluated (because its expression is not printed to the screen).

In [3]:
L = Lazy

x = L(2) + L(3) * L(10)
y = L(200) * L(3)
unused1 = L(500) + L(-200)

z = x + y
unused2 = unused1 + x

print(x)
print(z)

Lazy expression: (2) + ((3) * (10))
32
Lazy expression: ((2) + ((3) * (10))) + ((200) * (3))
632


Note that our client code can be very complex, and still we would only evaluate expressions as we need them. For example, even if some variables were used to create other variables, if those variables themselves are not used, then even the first expressions are not evaluated. Additionally, the code may dynamically use some variables and not others, for example, because of a loop or if condition. Even in these cases, only the used expressions will be evaluated.

In [4]:
from random import random

L = Lazy

unused1 = L(50) * L(3)
x = L(2) + L(3) * L(10)
y = L(200) * L(3)
z = L(20) + L(22) * L(2)
unused2 = unused1 + L(3)
t1 = x + y
t2 = x + z
unused3 = unused1 * z

print(x)

if random() < 0.5:
    print(t1, 't1')
else:
    print(t2, 't2')

Lazy expression: (2) + ((3) * (10))
32
Lazy expression: ((2) + ((3) * (10))) + ((20) + ((22) * (2)))
96 t2


If you run the example above several times, you will see that the program prints t1 or t2 randomly. Whenever one is selected by the if statement for printing, the other is not evaluated. The same is true of the unused variables, even though it may appear as if unused1 is used by unused2.

One minor disadvantage is that a variable that is used by several others is evaluated every time any of them is used. Note how x = 2 + 3 * 10 is evaluated by the first print statement, and again by the second.

This can be fixed by extending our Lazy class so that it stores the evaluated value and reuses it in subsequent evaluations. Note that this extension can be performed in a variety of ways, since our embedded Lazy class can be mixed with any python code we would like. Furthermore, the extension can be done by different developers in a different library or code base.

In [5]:
class LazyOptimized(Lazy):
    def __str__(self):
        return str(self.eval())

    def eval(self, debug=True):
        # store the value and then return it
        self.value = super().eval(debug=debug)
        return self.value

L2 = LazyOptimized

x = L2(2) + L2(3) * L2(10)
y = x + L2(500)
z = x * L2(100)

# note that x = 2 + 3 * 10 is evaluated only once!
print(z)
print(y)
print(x)

Lazy expression: ((2) + ((3) * (10))) * (100)
3200
Lazy expression: (32) + (500)
532
Lazy expression: 32
32


# Optimizing Code via Static Analysis

We can implement our desired optimization with a different approach: we can parse the code, analyze the resulting AST to determine which variables are unused, and remove those from the code. Fortunately, python provides us with libraries to do all of this. In particular, the ast and astor modules, which we have seen in previous lectures.

Note that this is not an embedding, but it is a good place to start to understand external embeddings.

Our analysis code works like this: first it finds all variables that are passed to a print statement. We call these **useful variables**. Then, it finds assignments to these variables, and marks every variable used in an assignment expression as a useful variable as well. The analysis code repeats these iterations until no further variables are identified as useful. Any other variables not marked as useful can be removed.

Below is the main entry point to perform this analysis.

In [6]:
import inspect
import ast
import astor

# Main entry point
def remove_useless(f):
    # parse source code of function f
    t = ast.parse(inspect.getsource(f))  

    # find useful variables, initially these are the ones passed to print()
    prev_variables = set()
    useful_variables = find_all_print_variables(t)
  
    # iterate until no more useful variables are identified
    while prev_variables != set(useful_variables):
        prev_variables = set(useful_variables)
        # find all statements that assign values to the useful variables
        # and return all variables in these statements (plus old useful variables)
        useful_statements, useful_variables = find_useful_statements(t, useful_variables)

    t.body[0].body = useful_statements
    return astor.to_source(t)

Our main function makes two function calls: find_all_print_variables and find_useful_statements. The source for these functions is shown below.

The first walks the parsed ast looking for AST nodes representing function calls, then finds the ones that calls print(). It then walks the arguments to every one of these calls and stores all variables in it, and returns that set of variables.

The second function takes a set of variable names as a parameter. It loops over all the statements in function f, and finds assignment statements. It checks if the assignment statements assign expressions to one of the variables in the set. If so, it markes the statement as useful, and adds all variables in the assignment statement to the set.

In [7]:
def find_all_print_variables(t):
    all_args = []
    for node in ast.walk(t):
        if isinstance(node, ast.Call) and isinstance(node.func, ast.Name): # if function call by function name
                if node.func.id == 'print':    # if the function being called is print!
                    all_args += node.args

    all_variables = []
    for arg in all_args:
        for node in ast.walk(arg):
            if isinstance(node, ast.Name):
                all_variables.append(node.id)
    return all_variables


def find_useful_statements(t, useful_variables):
    new_statements = []
    for statement in t.body[0].body:
        if not isinstance(statement, ast.Assign):
            new_statements.append(statement)
            continue

        if any([node.id in useful_variables for node in statement.targets if isinstance(node, ast.Name)]):
            new_statements.append(statement)
            for node in ast.walk(statement.value):
                if isinstance(node, ast.Name):
                    useful_variables.append(node.id)

    return new_statements, useful_variables

Now that we have defined all these functions, we can use them to automatically optimize our code and remove all useless variables. 

In [8]:
def f():
    x = 2 + 3 * 10
    y = 200 * x # unused
    z = 10 + x
    t = 5 * 3 # unused
    print(z, z + x)

# statically remove useless variables
code = remove_useless(f)
f = None

print(code)
exec(code)
f()

def f():
    x = 2 + 3 * 10
    z = 10 + x
    print(z, z + x)

42 74


Our static analysis code is not very intelligent. As it considers any variable passed to print to be useful, even when the print statement is not reachable (e.g. in an if statement). Also, our code can be confused by things like global variables and functions with side effects. For example, a function call may look useless (because its return value is not assigned to a useful variable), but it could assign values to useful global variables inside it.

In [9]:
def f():
    x = 2 + 3 * 10
    y = 200 * x # unused
    z = 10 + x
    t = 5 * 3 # unused
    print(x)
    if False:
        print(z) # clearly useless but our code wont know

# statically remove useless variables
code = remove_useless(f)
f = None

print(code)
exec(code)
f()

def f():
    x = 2 + 3 * 10
    z = 10 + x
    print(x)
    if False:
        print(z)

32


## Tradeoffs

There are several tradeoffs to be observed between these two solutions, many of which can be generalized to deep embeddings and external embeddings in general.

Client code:
1. The static analysis does not require the client code to use any special types or operators, while the deep embedding requires client code use the Lazy class.
2. The deep embedding client code still creates Lazy objects for useless operators, even if they are not performed, while the static analysis output code does not. In a sense, the static analysis output code is more efficient since it uses less space and time. If our operations were not + and \*, but were something very expensive instead (like a call to a machine learning or graph algorithm), then this added overhead in the deep embedding solution will be negligible compared to the optimization.

Functionality:
1. The deep embedding solution is perfectly accurate, it only evaluates things that need to be evaluation, this comes at a cost of performance: since the embedding creates a Lazy object for every expression, including useless ones.
2. The static analysis solution may have bad accuracy. It cannot tell with certainty which variables are useless and which are not in the general case. The solution must therefore be conservative, if it is uncertain, it must assume the variable under consideration is useful. The smarter the analysis algorithm is the better the accuracy will be, but there are theoretical limits, as this problem is undecidable.

Implementation:
1. The deep embedding solution expresses the functionality directly, and is closer to the style of typical programming developers are usually engaged in.
2. The static analysis solution is more indirect and works in a way that is less familiar. The complexity of that solution depends not only on the complexity of the task at hand, but also the complexity of the syntax of the language, as well as its semantics and types etc.

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">

<h1> External Embeddings </h1>

</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<p>
An external embedding works similar to the static analysis solution above, but with an extra component. The static analysis solution produces an output that is stand alone. It is optimized code that can be run directly.
</p>

<p>
An external embedding is a bit different. It has a static analysis component that may analyze code and transform it, but it does so with the purpose of producing embedding code. That code may be a shallow or a deep embedding, and is can be used by other components and programs as if it was directly embedded in them.
</p>

<p>
An example of an external embedding is the XML layout language in Android Development. To simplify layout design, Android allows users to use an XML language to specify layouts. Every layout is specified in its own file. Androind can be told to use an XML file to populate the UI using an API call.
</p>

<p>
The API call parses the XML code and uses it to populate the actual UI, it also creates a collection of Java objects that represent the UI elements, and that can be manipulated from the Java code.
</p>

<p>
HTML and JavaScript form a similar embedding. HTML code is parsed by the browser and used to create the UI, but it is also parsed to create the DOM, which is a collection of objects and methods representing the UI elements and its manipulation. DOM has both shallow (direct functions and properties) and deep (handlers, event listeners, and watchers of various forms) embedded components. JQuery adds another layer of embedding on top of the DOM.
</p>

<p>
External Embeddings exihibt some of the tradeoffs mentioned above when compared to shallow and deep embedding. However, they are very powerful, as they allow the AST of the language itself to be extended and modified, and allows transformations and analysis to be performed at various stages, including during parsing of the new language, static analysis of the new language, and during runtime via the generated embedded code.
</p>

</div>